<a href="https://colab.research.google.com/github/inclinedadarsh/dl-assignments/blob/main/dl-5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN vs MLP on MNIST - Minimal PyTorch Notebook

## 1. Imports & Device Setup

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


## 2. Data Loading + Augmentation

In [3]:
# Basic augmentation + normalization
transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

100%|██████████| 9.91M/9.91M [00:01<00:00, 6.06MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 160kB/s]
100%|██████████| 1.65M/1.65M [00:05<00:00, 318kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 13.0MB/s]


## 3. MLP Model (Baseline)

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

mlp = MLP().to(device)

## 4. CNN Model

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)  # 28x28 -> 28x28
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) # 14x14 -> 14x14
        self.fc1 = nn.Linear(32*7*7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.pool = nn.MaxPool2d(2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 28 -> 14
        x = self.pool(F.relu(self.conv2(x)))  # 14 -> 7
        x = x.view(-1, 32*7*7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

cnn = CNN().to(device)

## 5. Training Function

In [6]:
def train_model(model, name):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    epochs = 5  # minimal training

    print(f"\nTraining {name}...")
    for epoch in range(epochs):
        total_loss = 0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, {name} Loss: {total_loss/len(train_loader):.4f}")

    return model

## 6. Evaluation Function

In [7]:
def evaluate(model, name):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
    acc = 100. * correct / len(test_loader.dataset)
    print(f"{name} Test Accuracy: {acc:.2f}%")
    return acc

## 7. Train & Compare

In [8]:
# Train MLP
mlp = train_model(mlp, "MLP")

# Train CNN
cnn = train_model(cnn, "CNN")

# Evaluate both
print("\n" + "="*40)
acc_mlp = evaluate(mlp, "MLP")
acc_cnn = evaluate(cnn, "CNN")
print("="*40)
print(f"Final: MLP {acc_mlp:.2f}% vs CNN {acc_cnn:.2f}%")


Training MLP...
Epoch 1/5, MLP Loss: 0.2919
Epoch 2/5, MLP Loss: 0.1321
Epoch 3/5, MLP Loss: 0.1003
Epoch 4/5, MLP Loss: 0.0843
Epoch 5/5, MLP Loss: 0.0749

Training CNN...
Epoch 1/5, CNN Loss: 0.1816
Epoch 2/5, CNN Loss: 0.0619
Epoch 3/5, CNN Loss: 0.0456
Epoch 4/5, CNN Loss: 0.0375
Epoch 5/5, CNN Loss: 0.0323

MLP Test Accuracy: 97.10%
CNN Test Accuracy: 98.81%
Final: MLP 97.10% vs CNN 98.81%


## Summary
- **CNN** > **MLP** on MNIST (even with minimal layers)
- Data augmentation helps generalization
- CNN learns spatial patterns → better for images

**Done!** Copy each block into Jupyter Lab.  
Expected: **MLP ~98%**, **CNN ~99%+** after 5 epochs.